In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.optimize import minimize
#import pythran

import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
url_region_target = {
    'global-confirmed' : 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'global-fatalities' : 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'global-recovered' : 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',
    'US-confirmed' : 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
    'US-fatalities' : 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
}

In [ ]:
def data_processor_jhu(csv_url, Target):
    df = pd.read_csv(csv_url)
    df['Province/State'].fillna("",inplace = True)
    df['Country_Region'] = df['Country/Region'] + ' ' + df['Province/State']
    df['Country_Region'] = df['Country_Region'].apply(lambda x : x.rstrip())
    df = pd.concat([df['Country_Region'], df.iloc[:, 4:-1]], axis=1)
    df = df.melt(id_vars='Country_Region').rename(columns={'variable':'Date', 'value': Target})
    df['Date'] = pd.to_datetime(df['Date'])
    return df

def data_processor_jhu_US(csv_url, Target):
    df = pd.read_csv(csv_url)
    df['Admin2'].fillna("",inplace = True)
    df['Country_Region'] = df['Country_Region'] + ' ' + df['Province_State'] + ' ' + df['Admin2']
    df['Country_Region'] = df['Country_Region'].apply(lambda x : x.rstrip())
    if Target == 'ConfirmedCases':
        df = pd.concat([df['Country_Region'], df.iloc[:, 11:]], axis=1)
    else:
        df = pd.concat([df['Country_Region'], df.iloc[:, 12:]], axis=1)
    df = df.melt(id_vars='Country_Region').rename(columns={'variable':'Date', 'value': Target})
    df['Date'] = pd.to_datetime(df['Date'])
    return df

In [ ]:
df_global_confirmed = data_processor_jhu(url_region_target['global-confirmed'], 'ConfirmedCases')
df_global_confirmed = df_global_confirmed.sort_values(by=['Country_Region', 'Date'])
df_global_fatalities = data_processor_jhu(url_region_target['global-fatalities'], 'Fatalities')
#df_global_recovered = data_processor_jhu(url_region_target['global-recovered'], 'Recovered')
df_global = df_global_confirmed.merge(df_global_fatalities, left_on=['Country_Region', 'Date'], right_on=['Country_Region', 'Date'])
#df_global = df_global_confirmed.merge(df_global_recovered, left_on=['Country_Region', 'Date'], right_on=['Country_Region', 'Date'])

In [ ]:
df_us_confirmed = data_processor_jhu_US(url_region_target['US-confirmed'], 'ConfirmedCases')
df_us_confirmed = df_us_confirmed.sort_values(by=['Country_Region', 'Date'])
df_us_fatalities = data_processor_jhu_US(url_region_target['US-fatalities'], 'Fatalities')
df_us = df_us_confirmed.merge(df_us_fatalities, left_on=['Country_Region', 'Date'], right_on=['Country_Region', 'Date'])

In [ ]:
# No percentage change
df_raw = pd.concat([df_global, df_us], ignore_index=True).sort_values(by=['Country_Region', 'Date'])
# df_raw['confirmed_diFF'] = df_raw.ConfirmedCases.groupby(df_raw.Country_Region).diff() 
# df_raw['fatalities_diFF'] = df_raw.Fatalities.groupby(df_raw.Country_Region).diff() 
# df_raw = df_raw.dropna()
df_raw

"Synching" with Kaggle's data for that juicy population column:

In [ ]:
url_kaggle = "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/20475/1267893/compressed/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1592319944&Signature=HqEVwXWYMMOl35YrhvdluYBSUsZ92qBmbZXw6fSeqmOQ8p3ElJLOgW31kLwdgrSueNx4LMZhqD8dGXmwUfesKSbikdMxV1lQ7z4bwRBy1IO4DHhOhvthsjb8Mog%2BBEkQHnML%2BQDC81bfXVubZsM6S4pfwHgfwktyIkfBDswJUe38UgddsUljhrLVorMXe%2BUEpc6dbNnf5BJlnE7zQlRCrjHOw3jQKismQF%2F%2FbArHunLJsB9%2BKPs%2B10r30RU1OVY6j12tK04w41gHlCo1XWWbrArXgpruklygtOXDjpzr%2Fu8Pe%2BRi6TYwfEX0fUz6Vcwbo2WwSrdf0k3VTV0RQFnZ3w%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip"

In [ ]:
df_kaggle = pd.read_csv(url_kaggle)
df_kaggle['Date'] = pd.to_datetime(df_kaggle['Date'])
df_kaggle['County'].fillna("",inplace = True)
df_kaggle['Province_State'].fillna("",inplace = True)
df_kaggle['Country_Region'] = df_kaggle['Country_Region'] + ' ' + df_kaggle['Province_State'] + ' ' + df_kaggle['County']
df_kaggle['Country_Region'] = df_kaggle['Country_Region'].apply(lambda x : x.rstrip())
df_kaggle.drop(columns=['County', 'Province_State'], inplace = True)

df_kaggle

In [ ]:
#temp_df = pd.DataFrame({'ConfirmedCases':df_kaggle['TargetValue'].iloc[::2].values, 'Fatalities':df_kaggle['TargetValue'].iloc[1::2].values})
temp_df = df_kaggle[df_kaggle.Target == 'ConfirmedCases'].drop(columns='Target').rename(columns={"TargetValue" : "confirmed_diff"})
temp_df_2 = df_kaggle[df_kaggle.Target == 'Fatalities'].drop(columns=['Id', 'Target', 'Weight', 'Population']).rename(columns={"TargetValue" : "fatalities_diff"})
df_kaggle_final = temp_df.merge(temp_df_2, left_on=['Country_Region', 'Date'], right_on=['Country_Region', 'Date'])
df_kaggle_final

In [ ]:
# Combine everything
df_all = df_kaggle_final.merge(df_raw, left_on=['Country_Region', 'Date'], right_on=['Country_Region', 'Date'])
df_all

In [ ]:
# Delete unused dfs
del temp_df, temp_df_2, df_kaggle_final, df_kaggle, df_raw, df_global_confirmed, df_global_fatalities, df_global, df_us_confirmed, df_us_fatalities, df_us

# Setting up the model - SIRD (Susceptible-Infected-Recovered-Dead)

Details [here](https://www.overleaf.com/read/tnyfyyvggnsv).

Using US New York for testing run:

In [ ]:
df_nyc = df_all[df_all.Country_Region == 'US New York New York']
df_nyc

For the sake of simplicity, assume $S(0) = N - I(0)$. Also take, $I(0)$ to be the first non-zero confirmed cases number.

In [ ]:
df_nyc_train = df_nyc[df_nyc.ConfirmedCases > 0]
df_nyc_train

Getting some initial condition from the dataframe:

In [ ]:
N = df_nyc_train.Population.iloc[0]
#N = 850
#N = 1
I_0 = df_nyc_train.ConfirmedCases.iloc[0]
#I_0 = df_nyc_train.ConfirmedCases.iloc[0]/df_nyc_train.Population.iloc[0]
#I_0 = 0.001
S_0 = N - I_0
R_0 = 0
D_0 = N - I_0 - S_0 - R_0

We want $\delta t$ to be small (say a quarter of a day):

In [ ]:
T = 1; 
r = 2**10; 
n = (df_nyc_train.shape[0]-1)*r;
dt = T/n; 
Dt = r*dt; 
M = int(n/r);

We need tree sources of Gaussian noise, and let the number of sims be 100, so we are going to create 3 standard normal 100 by M matrices:

In [ ]:
def SIRD_Winc():
    T = 1; 
    r = 2**10; 
    n = (df_nyc_train.shape[0]-1)*r;
    dt = T/n; 
    Dt = r*dt; 
    M = int(n/r);
    eta_1 = np.random.normal(0, 1, (5000, n)) * np.sqrt(dt)
    eta_1 = np.add.reduceat(eta_1, np.arange(0, n, r), axis = 1)
    eta_2 = np.random.normal(0, 1, (5000, n)) * np.sqrt(dt)
    eta_2 = np.add.reduceat(eta_2, np.arange(0, n, r), axis = 1)
    eta_3 = np.random.normal(0, 1, (5000, n)) * np.sqrt(dt)
    eta_3 = np.add.reduceat(eta_3, np.arange(0, n, r), axis = 1)
    Eta = [eta_1, eta_2, eta_3]
    return Eta

Etas = SIRD_Winc()

Test with some dummy parameters:

In [ ]:
#7.52107377e+01, 2.28947946e+01, 6.96504586e-02, 8.33582662e+06
beta = 7.52107377e+01
gamma = 2.28947946e+01
mu =  6.96504586e-02

In [ ]:
S = np.zeros((1000, M+1))
I = np.zeros((1000, M+1))
R = np.zeros((1000, M+1))
D = np.zeros((1000, M+1))

#S[:,0] = S_0
S[:,0] = 8.33582662e+06
I[:,0] = I_0
R[:,0] = R_0
D[:,0] = D_0

for i in range(1, M+1):
    S[:, i] = S[:, i-1] - (beta * I[:, i-1] * S[:, i-1]/N) * Dt + np.sqrt(beta * I[:, i-1] * S[:, i-1]/N) * eta_1[:,i-1]
    #S[S[:, i] > N] = N
    I[:, i] = I[:, i-1] + (beta * I[:, i-1] * S[:, i-1]/N - (gamma + mu) * I[:, i-1]) * Dt - np.sqrt(beta * I[:, i-1] * S[:, i-1]/N) * eta_1[:,i-1] + \
              np.sqrt((gamma + mu)*I[:, i-1]) * eta_2[:,i-1]
    I[I[:, i] < 0] = 0
    R[:, i] = R[:, i-1] + gamma * I[:, i-1] * Dt - gamma * np.sqrt((gamma + mu) * I[:, i-1]) /(gamma + mu) * eta_2[:,i-1] + \
              np.sqrt(gamma*mu*I[:, i-1]/(gamma + mu)) * eta_3[:,i-1]
    R[R[:, i] < 0] = 0
#     S[:, i] = S[:, i-1] - (beta * I[:, i-1] * S[:, i-1]) * dt + np.sqrt(beta * I[:, i-1] * S[:, i-1]) * np.sqrt(dt) * eta_1[:,i-1]
#     I[:, i] = I[:, i-1] + (beta * I[:, i-1] * S[:, i-1] - (gamma + mu) * I[:, i-1]) * dt - np.sqrt(beta * I[:, i-1] * S[:, i-1]) * np.sqrt(dt) * eta_1[:,i-1] + \
#               np.sqrt((gamma + mu)*I[:, i-1]) * np.sqrt(dt) * eta_2[:,i-1]
#     R[:, i] = R[:, i-1] + gamma * I[:, i-1] * dt - gamma * np.sqrt((gamma + mu) * I[:, i-1]) /(gamma + mu) * np.sqrt(dt) * eta_2[:,i-1] + \
#               np.sqrt(gamma*mu*I[:, i-1]/(gamma + mu)) * np.sqrt(dt) * eta_3[:,i-1]
    D[:, i] = N - S[:, i] - I[:, i] - R[:, i]

In [ ]:
x = range(101)
plt.plot(x, np.mean(D[:, 0:],axis=0).T, label='Modeled Fitted Fatalities');
plt.plot(x, df_nyc_train.Fatalities, label='US NYC Fatalities');
plt.legend()

In [ ]:
plt.plot(x, np.mean(R[:, 0:],axis=0).T, label = 'R');
plt.plot(x, np.mean(I[:, 0:],axis=0).T, label = 'I');
plt.plot(x, np.mean(S[:, 0:],axis=0).T, label = 'S');
plt.legend()

In [ ]:
plt.plot(x, D[:, 0:].T, label = 'S');

In [ ]:
def SIRD(sims_no, days, pop, init_inf, params, winc):
    # sims_no: number of simulations, recommended 10000 at least
    # dt: size of time steps - in fraction of a day
    # params: beta, gamma, mu, S_0
    
    beta = params[0]
    gamma = params[1]
    mu = params[2]
    N = pop
    
    T = 1; 
    r = 2**10; 
    #n = (df_nyc_train.shape[0]-1)*r;
    n = days*r
    dt = T/n; 
    Dt = r*dt; 
    M = int(n/r);
    
#     eta_1 = np.random.normal(0, 1, (sims_no, n)) * np.sqrt(dt)
#     eta_1 = np.add.reduceat(eta_1, np.arange(0, n, r), axis = 1)
#     eta_2 = np.random.normal(0, 1, (sims_no, n)) * np.sqrt(dt)
#     eta_2 = np.add.reduceat(eta_2, np.arange(0, n, r), axis = 1)
#     eta_3 = np.random.normal(0, 1, (sims_no, n)) * np.sqrt(dt)
#     eta_3 = np.add.reduceat(eta_3, np.arange(0, n, r), axis = 1)
    
    eta_1 = winc[0]
    eta_2 = winc[1]
    eta_3 = winc[2]
    
    S = np.zeros((sims_no, M+1))
    I = np.zeros((sims_no, M+1))
    R = np.zeros((sims_no, M+1))
    D = np.zeros((sims_no, M+1))
    
    S_0 = params[3]
    I_0 = init_inf
    R_0 = 0
    D_0 = 0
    
    S[:,0] = S_0
    I[:,0] = I_0
    R[:,0] = R_0
    D[:,0] = D_0
    
    for i in range(1, M+1):
        S[:, i] = S[:, i-1] - (beta * I[:, i-1] * S[:, i-1]/N) * Dt + np.sqrt(beta * I[:, i-1] * S[:, i-1]/N) * eta_1[:,i-1]
        S[S[:, i] < 0] = 0
        I[:, i] = I[:, i-1] + (beta * I[:, i-1] * S[:, i-1]/N - (gamma + mu) * I[:, i-1]) * Dt - np.sqrt(beta * I[:, i-1] * S[:, i-1]/N) * eta_1[:,i-1] + \
                  np.sqrt((gamma + mu)*I[:, i-1]) * eta_2[:,i-1]
        I[I[:, i] < 0] = 0
        R[:, i] = R[:, i-1] + gamma * I[:, i-1] * Dt - gamma * np.sqrt((gamma + mu) * I[:, i-1]) /(gamma + mu) * eta_2[:,i-1] + \
                  np.sqrt(gamma*mu*I[:, i-1]/(gamma + mu)) * eta_3[:,i-1]
        R[R[:, i] < 0] = 0
        D[:, i] = N - S[:, i] - I[:, i] - R[:, i]
        D[D[:, i] < 0] = 0
    return S, I, R, D

def params_optim(df):
    pop = df.Population.iloc[0]
    deaths_time_series = df.Fatalities
    initial_infected = df.ConfirmedCases.iloc[0]
    
    T = 1;
    r = 2**10; 
    n = (df.shape[0]-1)*r
    dt = T/n; 
    Dt = r*dt; 
    M = int(n/r);
    sims = 5000

    winc_1 = np.random.normal(0, 1, (sims, n)) * np.sqrt(dt)
    winc_1 = np.add.reduceat(winc_1, np.arange(0, n, r), axis = 1)
    winc_2 = np.random.normal(0, 1, (sims, n)) * np.sqrt(dt)
    winc_2 = np.add.reduceat(winc_2, np.arange(0, n, r), axis = 1)
    winc_3 = np.random.normal(0, 1, (sims, n)) * np.sqrt(dt)
    winc_3 = np.add.reduceat(winc_3, np.arange(0, n, r), axis = 1)
    
    Winc = [winc_1, winc_2, winc_3]

    def target_func(params):
        _, _, _, D= SIRD(sims, df.shape[0]-1, pop, initial_infected, params, Winc)
        D_mean = np.mean(D,axis=0) 
        return np.sum((deaths_time_series - D_mean)**2)
    
    x0 = np.array([40, 20, 1, pop])
    bnds = ((0.0000001, None), (0.0000001, None), (0.0000001, None), (0.0000001, pop))
    res = minimize(target_func, x0, bounds=bnds, options={'ftol': 1e-10, 'disp': True, 'maxiter' : 20000})
    
    return res
    

In [ ]:
results = params_optim(df_nyc_train)

In [ ]:
results

In [ ]:
results

In [ ]:
S,I,R,D = SIRD(5000, df_nyc_train.shape[0]-1, df_nyc_train.Population.iloc[0], df_nyc_train.ConfirmedCases.iloc[0], results.x, Etas)